In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

dataset = load_dataset("cnn_dailymail", '1.0.0')
dataset

c:\Users\Tairo Kageyama\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Tairo Kageyama\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [5]:
dataset = dataset['validation'][:3]

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [10]:
import evaluate
rouge = evaluate.load('rouge')
summaries = []
for data in dataset["article"]:
    print(data)
    test = tokenizer(data, return_tensors="pt", max_length=500, truncation=True, padding="max_length")

    generated_summaries = model.generate(test['input_ids'], max_length=38, min_length=12, length_penalty=1.0, num_beams=4, early_stopping=True)
    print(generated_summaries)
    summary = tokenizer.decode(generated_summaries[0], skip_special_tokens=True)
    print(summary)
    summaries.append(summary)




(CNN)Share, and your gift will be multiplied. That may sound like an esoteric adage, but when Zully Broussard selflessly decided to give one of her kidneys to a stranger, her generosity paired up with big data. It resulted in six patients receiving transplants. That surprised and wowed her. "I thought I was going to help this one person who I don't know, but the fact that so many people can have a life extension, that's pretty big," Broussard told CNN affiliate KGO. She may feel guided in her generosity by a higher power. "Thanks for all the support and prayers," a comment on a Facebook page in her name read. "I know this entire journey is much bigger than all of us. I also know I'm just the messenger." CNN cannot verify the authenticity of the page. But the power that multiplied Broussard's gift was data processing of genetic profiles from donor-recipient pairs. It works on a simple swapping principle but takes it to a much higher level, according to California Pacific Medical Center 

In [12]:
dataset['highlights']

['Zully Broussard decided to give a kidney to a stranger . A new computer program helped her donation spur transplants for six kidney patients .',
 'The 20th MLS season begins this weekend . League has changed dramatically since its inception in 1996 . Some question whether rules regarding salary caps and transfers need to change .',
 'Bafetimbi Gomis collapses within 10 minutes of kickoff at Tottenham . But he reportedly left the pitch conscious and wearing an oxygen mask . Gomis later said that he was "feeling well" The incident came three years after Fabrice Muamba collapsed at White Hart Lane .']

In [13]:
results = rouge.compute(predictions=[summary], references=[dataset['highlights']])
print(results)

{'rouge1': 0.27692307692307694, 'rouge2': 0.09523809523809523, 'rougeL': 0.2153846153846154, 'rougeLsum': 0.2153846153846154}
